In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [19]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [20]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [21]:
y_pred

array([14.53986486, 13.74042222, 15.59333908, ..., 15.83492293,
       16.78317605, 19.65462607])

In [9]:
import numpy as np

In [23]:
np.mean(y_pred)

16.191691679979066

In [24]:
year = 2021
month = 2
#df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [25]:
df_result = pd.DataFrame()
df_result['prediction'] = y_pred
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df_result.index.astype('str')
df_result

,prediction,ride_id
0,14.539865,2021/02_0
1,13.740422,2021/02_1
2,15.593339,2021/02_2
3,15.188118,2021/02_3
4,13.817206,2021/02_4
...,...,...
990108,12.433246,2021/02_990108
990109,12.910885,2021/02_990109
990110,15.834923,2021/02_990110
990111,16.783176,2021/02_990111


In [27]:
df_result.to_parquet(
    'output.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)